# 範例

取得 SIFT 特徵

- 轉成灰階圖片
- 需要額外安裝 OpenCV 相關套件

## SIFT

**SIFT 是一種表徵 (appearance feature)**

基於局部的外觀特徵，進一步考慮到圖片特徵的狀況

* 縮放不變性，旋轉不變性
* 光線與雜訊容忍度高

在 SIFT 演算法中可以了解到如何做關鍵點偵測，並如何抽出 SIFT 特徵來敘述關鍵點

### 尺度不變性

SIFT 主要考慮到的問題之一是尺度

以 corner detector (e.g. Harris) 為例，Filter 可以偵測到範圍以內的角落點，

但是同樣的 pattern 放大後以同樣的 Filter 去偵測就會失敗~ 以為是直線了

![image](https://docs.opencv.org/3.4/sift_scale_invariant.jpg)

SIFT 會基於邊緣檢測抽取特徵，但不是使用前面提過的 Sobel，概念上是 LoG 但是考慮到計算量使用 DoG 做邊緣檢測

Laplacian of Gaussian (LoG)

* 先對圖片做 Gaussian Blur 再算二階導數取得邊緣

Difference of Gaussian (DoG)

* 圖片經過不同程度的縮放後計算出不同程度的 Gaussian Blur 最後合併得到一個 Gaussian Pyramid，其差值即為 DoG。結果可以視為 LoG 的約略值 (沒有做二階導數)

![image](https://docs.opencv.org/3.4/sift_dog.jpg)

(p.s. 這邊討論的特徵主要是物體的邊緣，而二階導數是個適合的工具來找出邊緣，因此這邊才會以此討論 LoG 與 DoG)

### 極值偵測

DoG 影像包含多種尺度

接著要針對每個 pixel 判斷是否為極值

判斷範圍 8+18 = 26
* 自己本身周遭的 8 個 pixel
* 同一個 scale 圖片但不同模糊尺度相鄰位置共 9*2=18 個 pixel

假如該 pixel 為判斷範圍內的最大/最小值，則將其設為有興趣的關鍵點

![image](https://docs.opencv.org/3.4/sift_local_extrema.jpg)

### 關鍵點定位

經過多尺度極值偵測之後，會得到許多候選的關鍵點，其中也包含許多噪音跟邊的關鍵點，需要更進一步根據周遭資訊來修正並過濾關鍵點。

鄰近資料差補:主要根據相鄰資訊來修正極值的位置

過濾不明顯關鍵點:根據計算曲率來判斷是否為不明顯的關鍵點

過濾邊緣關鍵點:根據計算曲率來判斷是否為不明顯的關鍵點

### 方位定向

前面我們定義並過濾了許多關鍵點，但是關鍵點只有包含尺度跟位置

SIFT 還想要保有旋轉不變性，因此要給關鍵點定義一個方向

- 以每 10 度為單位計算周圍的梯度值
- 梯度值最大的方向當作是該關鍵點的主要方向

### 關鍵點描述子

賦與關鍵點位置，尺度，方向確保移動，縮放，旋轉的不變性，還需要額外建立描述子來確保不同光線跟視角也有不變性

- 描述子會正規化成 128 維的特徵向量
- 以關鍵點周圍 16\*16 的區域共 4\*4 的子區域，計算 8 個方向的直方圖，共 4\4\*8 = 128 維的特徵向量


Note：每個關鍵點都會產生 128 維的特徵，而圖片會產生 N 個關鍵點，也就是會產生 (N, 128) 維度特徵

In [1]:
import numpy as np
import cv2

img = cv2.imread('lena.png')
print('OpenCv Version:',cv2.__version__)

OpenCv Version: 4.5.1


In [3]:
# 轉灰階圖片
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 建立 SIFT 物件
SIFT_detector = cv2.SIFT_create()

# 取得 SIFT 關鍵點位置
keypoints = SIFT_detector.detect(img_gray, None)

#　畫圖 + 顯示圖片
img_show = cv2.drawKeypoints(img_gray, keypoints, img)
while True:
    cv2.imshow('SIFT', img_show)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break